In [1]:
# eval_ckpt_example.py - single image inference for EfficientNet
# adapted from TF → PyTorch


# imports 
import os
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import torch.nn as nn

# project-specific imports
from ModelBuilder import efficientnet
from utils import load_model_weights  # optional: load pretrained weights


In [2]:

# device 
# adapted: TF device handling → PyTorch device handling
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cpu


In [3]:

#transforms 
# reused from main training/eval pipeline
# ensures preprocessing consistency with dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # match input size of B0 (adjustable for B1-B8, L2)
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # ImageNet normalization
        std=[0.229, 0.224, 0.225]
    )
])


In [4]:

# single image loader 
def load_image(img_path):
    """Load and preprocess a single image."""
    image = Image.open(img_path).convert('RGB')  # ensure 3 channels
    image = transform(image)
    return image.unsqueeze(0).to(device)  # add batch dimension


In [5]:

# model setup 
model_name = 'efficientnet-b0'  # can change to b0-b8 or l2
model = efficientnet(model_name=model_name)
print(f"Built model: {model_name}")

# adjust final layer for dataset
num_classes = 9  # number of classes in your dataset
in_features = model.fc.in_features  # input features to final linear layer
model.fc = nn.Linear(in_features, num_classes)  # replace final layer
model.to(device)
model.eval()  # set to evaluation mode

# optional: load pretrained weights if available
# weight_path = "path_to_pth_file.pth"
# if weight_path:
#     model = load_model_weights(model, weight_path, device)
# won't implement for now since this is a quick test

Built model: efficientnet-b0


EfficientNet(
  (stem_conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (stem_bn): BatchNorm2d(32, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
  (activation): Swish()
  (blocks): ModuleList(
    (0): MBConvBlock(
      (depthwise_conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn1): BatchNorm2d(32, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
      (se_block): SEBlock(
        (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
        (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
        (activation): Swish()
      )
      (project_conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
      (activation): Swish()
    )
    (1): MBConvBlock(
      (expand_conv): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn0): 

In [6]:

#  dummy run 
# dummy input for sanity check
x = torch.randn(1, 3, 224, 224).to(device)
y = model(x)
print("Output shape:", y.shape)  # expected: [1, 9]

# convert output logits to probabilities
probs = torch.softmax(y, dim=1)
print("Probabilities:", probs)
print("Sum of probabilities:", probs.sum())  # should be 1.0


Output shape: torch.Size([1, 9])
Probabilities: tensor([[0.1119, 0.1078, 0.1130, 0.1137, 0.1136, 0.1097, 0.1100, 0.1094, 0.1109]],
       grad_fn=<SoftmaxBackward0>)
Sum of probabilities: tensor(1.0000, grad_fn=<SumBackward0>)


In [7]:
#  prediction function 
def predict(img_path, class_names):
    """Run inference on one image and print predicted class + probability."""
    img_tensor = load_image(img_path)

    with torch.no_grad():
        output = model(img_tensor)
        probs = F.softmax(output, dim=1)
        pred_idx = torch.argmax(probs, dim=1).item()
        pred_prob = probs[0, pred_idx].item()

    print(f"\nImage: {img_path}")
    print(f"Predicted: {class_names[pred_idx]}  ({pred_prob:.4f})")

    return pred_idx, pred_prob


In [8]:

#  test prediction 
root_dir = r"C:\Users\nourh\OneDrive\Desktop\EffecientNetProject\Data\Fish_Dataset"
class_names = sorted(os.listdir(root_dir))  # get folder names as class labels

test_image_path = os.path.join(root_dir, class_names[0], "00001.png")
predict(test_image_path, class_names)  # untrained example; will be retrained later



Image: C:\Users\nourh\OneDrive\Desktop\EffecientNetProject\Data\Fish_Dataset\Black Sea Sprat\00001.png
Predicted: Red Mullet  (0.1137)


(3, 0.11371579021215439)